In [6]:
%run 'Load_base_data.ipynb'

In [7]:
import pandas as pd
fire_inspections = pd.read_csv('RawData/Fire_Inspections.csv', low_memory=False)

In [8]:
#get Lat Long
fire_inspections['Lat']=fire_inspections['Location'].apply(get_latitude)
fire_inspections['Long']=fire_inspections['Location'].apply(get_longitude)
fire_inspections['value'] =1

In [4]:
# convert to geo pandas
fire_inspections['Coordinates'] = list(zip(fire_inspections.Long, fire_inspections.Lat))
fire_inspections['Coordinates'] = fire_inspections['Coordinates'].apply(Point)

In [5]:
#remove data with no lat long
fire_inspections=fire_inspections.dropna(subset=['Lat','Long'])

In [6]:
fire_inspections_main_gdf = gpd.GeoDataFrame(fire_inspections, geometry='Coordinates')

In [7]:
#spatial join gps 
fire_inspections_main_parcels = gpd.sjoin(fire_inspections_main_gdf, geo_loc_gdf_sub, how="left", op='intersects')

In [8]:
fire_inspections_clean = pd.DataFrame(fire_inspections_main_parcels)

In [9]:
#code the status of inspections to parcel and then elongate so that each inspection type is its own column
fire_inspection_status =fire_inspections_clean[['Inspection Status','blklot','value']].groupby(['blklot','Inspection Status']).agg('count')
fire_inspection_status.reset_index(inplace=True)
fire_inspection_status.head()

fire_inspection_status_long=fire_inspection_status.pivot_table(index=['blklot'],columns='Inspection Status', values='value')
fire_inspection_status_long.fillna(0, inplace=True)
fire_inspection_status_long.reset_index(inplace=True)
fire_inspection_status_long.head()

Inspection Status,blklot,completed,expired,open/follow-up needed,pending
0,0004002,13.0,0.0,0.0,0.0
1,0005001,37.0,0.0,0.0,8.0
2,0006001,38.0,0.0,0.0,4.0
3,0007001,36.0,0.0,0.0,5.0
4,0010001,67.0,0.0,4.0,9.0


In [11]:
#code the referral agency to parcel and then elongate so that each referral agency is its own column
fire_inspection_referag =fire_inspections_clean[['Referral Agency','blklot','value']].groupby(['blklot','Referral Agency']).agg('count')
fire_inspection_referag.reset_index(inplace=True)
fire_inspection_referag.head()

fire_inspection_referag_long=fire_inspection_referag.pivot_table(index=['blklot'],columns='Referral Agency', values='value')
fire_inspection_referag_long.fillna(0, inplace=True)
fire_inspection_referag_long.reset_index(inplace=True)
fire_inspection_referag_long.head()

fire_inspections_main= fire_inspection_status_long.merge(fire_inspection_referag_long, left_on=['blklot'], 
                                                      right_on=['blklot'], how='left')

In [12]:
#code the inspection description to parcel and then elongate so that each inspection description is its own column
fire_inspection_insp_desc =fire_inspections_clean[['Inspection Type Description','blklot','value']].groupby(['blklot','Inspection Type Description']).agg('count')
fire_inspection_insp_desc.reset_index(inplace=True)
fire_inspection_insp_desc.head()


fire_inspection_insp_desc_long=fire_inspection_insp_desc.pivot_table(index=['blklot'],columns='Inspection Type Description', values='value')
fire_inspection_insp_desc_long.fillna(0, inplace=True)
fire_inspection_insp_desc_long.reset_index(inplace=True)
fire_inspection_insp_desc_long.head()


fire_inspections_main= fire_inspections_main.merge(fire_inspection_insp_desc_long, left_on=['blklot'], 
                                                      right_on=['blklot'], how='left')

In [15]:
#summarize numerical values per parcel
#Invoice Amount
#Fee
#Penalty Amount
#Posting Fee
#Paid Amount
#Interest Amount
fire_inspection_subset =fire_inspections_clean[['Invoice Amount',
                                           'Fee',
                                           'Penalty Amount',
                                           'Posting Fee',
                                           'Paid Amount',
                                           'Interest Amount',
                                           'blklot']].fillna(0)
fire_inspection_summary= fire_inspection_subset.groupby(['blklot']).agg('sum')
fire_inspection_summary.reset_index(inplace=True)
fire_inspection_summary.head()



fire_inspections_main= fire_inspections_main.merge(fire_inspection_summary, left_on=['blklot'], 
                                                      right_on=['blklot'], how='left')


In [17]:
R2Parcel = pd.read_csv('residential_units.csv')

In [18]:
#combine with residential data so that only save data for r2 buildings
fire_inspections_clean_test = R2Parcel.merge(fire_inspections_main,left_on=['Block and Lot Number'], 
                                                      right_on=['blklot'], how='inner')

In [19]:
#save clean dataset
fire_inspections_clean_test.to_csv('fire_inspections_clean.csv')

In [20]:
fire_inspections_clean_test.shape

(3305, 97)

In [22]:
#check for duplicates
print(fire_inspections_clean_test.shape)
print(fire_inspections_clean_test[['blklot']].drop_duplicates(['blklot']).shape)

(3305, 97)
(3305, 1)


### Inspections per year for data viz

In [21]:
#get summary per year for ploting 

fire_inspections['Inspection Start Date']=pd.to_datetime(fire_inspections['Inspection Start Date'],format='%m/%d/%Y')
fire_inspections['Year']=fire_inspections['Inspection Start Date'].apply(lambda x: x.year)
fire_inspections['Num_inspections']=1
fireinsp =fire_inspections[['Year','Num_inspections']].groupby('Year').agg(sum)
fireinsp.reset_index(inplace=True)
fireinsp.to_csv('fire_inspections_per_year.csv')